In [12]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz

In [13]:
aapl = yf.download(tickers="AAPL", period="7d", interval="1m")
aapl.reset_index(inplace=True)
aapl

[*********************100%***********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2024-03-20 09:30:00-04:00,175.729996,176.100006,175.500000,175.729996,175.729996,1350666
1,2024-03-20 09:31:00-04:00,175.710007,176.029999,175.669998,175.709900,175.709900,140912
2,2024-03-20 09:32:00-04:00,175.679993,175.819901,175.610001,175.729996,175.729996,176795
3,2024-03-20 09:33:00-04:00,175.725006,175.800003,175.630005,175.669998,175.669998,167278
4,2024-03-20 09:34:00-04:00,175.660004,175.660004,175.264008,175.307999,175.307999,203073
...,...,...,...,...,...,...,...
2722,2024-03-28 15:56:00-04:00,171.774994,171.860001,171.699997,171.835007,171.835007,257474
2723,2024-03-28 15:57:00-04:00,171.845001,171.990005,171.800003,171.945007,171.945007,255920
2724,2024-03-28 15:58:00-04:00,171.940002,171.949997,171.860001,171.860001,171.860001,292888
2725,2024-03-28 15:59:00-04:00,171.860001,171.899994,171.259995,171.520004,171.520004,1669914


In [14]:
data=pd.read_csv(r'C:\Users\Legion\Desktop\FinalYearProject\data\News.csv')
data

,Date,article_title,article,source_name,source_link
0,2024-03-29 08:41:00+00:00,Warren Buffett Is Raking In $4.36 Billion in A...,"For nearly 60 years, Berkshire Hathaway (NYSE:...",Yahoo Finance,https://finance.yahoo.com/quote/AAPL/news/warr...
1,2024-03-29 08:09:27+00:00,Huawei Profit Surges as It Takes Share From Ap...,(Bloomberg) -- Huawei Technologies Co. sustain...,Yahoo Finance,https://finance.yahoo.com/quote/AAPL/news/huaw...
2,2024-03-29 04:22:06+00:00,20 Countries that Receive the Most Foreign Dir...,"In this article, we will look at 20 countries ...",Yahoo Finance,https://finance.yahoo.com/quote/AAPL/news/20-c...
3,2024-03-29 01:33:10+00:00,20 Countries with the Lowest Homeless Populati...,"In this article, we will be navigating through...",Yahoo Finance,https://finance.yahoo.com/quote/AAPL/news/20-c...
4,2024-03-28 21:11:33+00:00,Q1 2024 Takeaways: The rise and fall of the Ma...,"As the first quarter of 2024 draws to a close,...",Yahoo Finance,https://finance.yahoo.com/quote/AAPL/video/q1-...
...,...,...,...,...,...
552,2024-03-05 16:59:20+00:00,Apple Stock Gloom Deepens as Pressure to Show ...,(Bloomberg) -- Apple Inc.’s move to shutter it...,Yahoo Finance,https://finance.yahoo.com/quote/AAPL/news/appl...
553,2024-03-05 16:55:49+00:00,What the shakeup in Magnificent 7 stocks means...,Stock market indices have been propped up on t...,Yahoo Finance,https://finance.yahoo.com/quote/AAPL/video/sha...
554,2024-03-05 16:46:00+00:00,Alphabet (GOOGL) Enhances Google Maps With New...,Alphabet’s GOOGL Google is gaining strong mome...,Yahoo Finance,https://finance.yahoo.com/quote/AAPL/news/alph...
555,2024-03-05 16:43:00+00:00,"Why Apple Stock Keeps Dropping, and What Could...","Apple stock is in the red this year, so Wall S...",Yahoo Finance,https://finance.yahoo.com/quote/AAPL/m/c76aec8...


In [15]:
def convertTimezone(timezone, df,name):
    """
    Converting timestamps in to a specified timezone.

    Arguments:
        timezone (str): Target timezone to convert timestamps to.
        df : DataFrame containing a column of timestamp strings.
        name: name of the timestamp column

    Returns a dataFrame with timestamps converted to the specified timezone.
    """
    # Convert timestamp column to datetime objects
    df[name] = pd.to_datetime(df[name])

    # Create timezone object for the target timezone
    target_tz = pytz.timezone(timezone)

    # Convert timestamps to the target timezone
    df[name] = df[name].dt.tz_convert(target_tz)

    return df

    

In [16]:
hist_df = convertTimezone('UTC', aapl,'Datetime')
hist_df

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2024-03-20 13:30:00+00:00,175.729996,176.100006,175.500000,175.729996,175.729996,1350666
1,2024-03-20 13:31:00+00:00,175.710007,176.029999,175.669998,175.709900,175.709900,140912
2,2024-03-20 13:32:00+00:00,175.679993,175.819901,175.610001,175.729996,175.729996,176795
3,2024-03-20 13:33:00+00:00,175.725006,175.800003,175.630005,175.669998,175.669998,167278
4,2024-03-20 13:34:00+00:00,175.660004,175.660004,175.264008,175.307999,175.307999,203073
...,...,...,...,...,...,...,...
2722,2024-03-28 19:56:00+00:00,171.774994,171.860001,171.699997,171.835007,171.835007,257474
2723,2024-03-28 19:57:00+00:00,171.845001,171.990005,171.800003,171.945007,171.945007,255920
2724,2024-03-28 19:58:00+00:00,171.940002,171.949997,171.860001,171.860001,171.860001,292888
2725,2024-03-28 19:59:00+00:00,171.860001,171.899994,171.259995,171.520004,171.520004,1669914


In [17]:
# get specific stock price at a certain day and time
# it will take as parameters the date of the published article, the number of minutes after the article was published

# i need to make sure that the stock prices dataset has no index.
# and i need to convert its timezone to UTC
# i need to make sure that the end date is 1 day ahead.



In [18]:
# label the data

In [19]:
def getStockPrice(ticker,date):
    # date_string = '2024-03-05 14:30:00+00:00'
    # format = '%Y-%m-%d %H:%M:%S%z'
    # date = datetime.strptime(date_string, format) # to pass as an argument

    end_date=date+timedelta(days=1)
    end_date=end_date.strftime("%Y-%m-%d %H:%M:%S%z").split()
    end_date=end_date[0]

    date= date.strftime("%Y-%m-%d %H:%M:%S%z").split()
    day=date[0]
    time=date[1].split("+")[0]

    data = yf.download(tickers=ticker, start=day, end=end_date, interval='1m', index_col=0)
    
    if data.empty:
        print(f"No data available for {ticker} on {date}")

    data_at_time = data.between_time(time, time)

    if not data_at_time.empty:
        stock_price = data_at_time['Close'].values[0]

    else:
        print(f"No data available for {ticker} on {date} at {time}")
    
    
    if stock_price is not None:
        print(f"The stock price of {ticker} on {day} at {time} was ${stock_price:.2f}")

In [20]:
# testing if the function works:
ticker = 'AAPL'
date = datetime(2024, 3, 5, 14, 30, 0)  # Assuming date is in UTC timezone with offset
getStockPrice(ticker, date)

[*********************100%***********************]  1 of 1 completed
The stock price of AAPL on 2024-03-05 at 14:30:00 was $170.07


In [21]:
ticker = 'AAPL'
date = datetime(2024, 3, 5, 14, 35, 0)  # Assuming date is in UTC timezone with offset
getStockPrice(ticker, date)

[*********************100%***********************]  1 of 1 completed
The stock price of AAPL on 2024-03-05 at 14:35:00 was $170.01
